# Experiments

### Setup

In [7]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [19]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [9]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG GOLDEN DATASET"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-b487a170' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=cc32d82e-cd85-4e39-95bb-1afc0942febb




10it [01:42, 10.27s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",1,2.233616,16aa87a2-57b8-433e-b963-e2542c3642af,30300786-9148-4240-bc0d-476ade01ada1
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,80.248481,1e9b198a-757d-4825-971d-37936c5bba30,ea89e8e2-38da-40ad-909a-f1cc7566d585
2,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,3.006039,341606a3-751a-41d4-8d13-2ae8cc502806,517ff81a-2f78-482a-922d-ea4d02fa857c
3,What testing capabilities does LangSmith have?,LangSmith offers testing capabilities that all...,None,LangSmith offers capabilities for creating dat...,1,2.009480,59544459-9d57-431e-b2ee-e09f37ee1312,30910edc-6747-4834-91fa-3d0b84606cb5
4,What is LangSmith used for in three sentences?,LangSmith is used for managing and running exp...,None,LangSmith is a platform designed for the devel...,1,2.317658,5e199ead-d9d8-4f7a-a3bc-8365798aedf7,fdaf9bbb-2612-4212-8a9c-76c6ebc7e14e
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,2.206141,5efce925-42f9-45a4-82f7-71b43f10611f,6d5d1319-b020-4be5-adbd-5301e9669204
6,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, simply...",None,To trace with the @traceable decorator in Pyth...,1,2.383511,61779256-72cc-45ad-a9fc-3471826ab829,f53bec42-21f5-44b0-bd14-c8601f125aa0
7,Does LangSmith support offline evaluation?,The provided context does not mention offline ...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.465332,6d392d5b-1b24-4332-87a1-4a696cd36c07,d5b86a3c-97ff-4465-b981-5c14dbf16d65
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily a platform for observab...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.236943,90675349-ce91-4d9c-aef9-b186912f90ba,916d72f8-44c7-407f-8c6d-7de7fd95cd6a
9,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, include th...",None,You can pass metadata with the @traceable deco...,1,3.013996,b256575c-e03c-44a4-bff6-64d4cbbeb65d,7f85bbae-7b8d-44b5-9e14-5d9eef78a99e


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [12]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-b04abaa4' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=e36f8ef0-4100-4931-ac10-57b3a57b351e




10it [00:20,  2.01s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,LangSmith supports online evaluation through i...,None,"Yes, LangSmith supports online evaluation as a...",0,2.543792,16aa87a2-57b8-433e-b963-e2542c3642af,00fb7962-31ad-463d-9b22-b9b99e5f81d1
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.924658,1e9b198a-757d-4825-971d-37936c5bba30,d8e9d141-48b3-4d61-99ea-a113a157b551
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.772262,341606a3-751a-41d4-8d13-2ae8cc502806,394a9162-78e7-4994-854b-e6dc9dd75d88
3,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.449454,59544459-9d57-431e-b2ee-e09f37ee1312,fa3e3271-fec9-48df-9e41-db6e6434c546
4,What is LangSmith used for in three sentences?,LangSmith is used to run experiments on a give...,None,LangSmith is a platform designed for the devel...,1,1.790081,5e199ead-d9d8-4f7a-a3bc-8365798aedf7,adc1adf6-fffb-401d-bbae-39c752049b8f
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.137246,5efce925-42f9-45a4-82f7-71b43f10611f,5b81d2f2-b8bb-4b5b-8f82-421e0af169a4
6,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,3.066721,61779256-72cc-45ad-a9fc-3471826ab829,e6da63b9-1e7a-4666-b6dc-da8c23901dd9
7,Does LangSmith support offline evaluation?,LangSmith supports online evaluations that pro...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.632255,6d392d5b-1b24-4332-87a1-4a696cd36c07,109e1d48-b996-40bb-9778-9b3a9b1fa62f
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.674006,90675349-ce91-4d9c-aef9-b186912f90ba,711edee0-89b9-4e74-8e13-d2c1f27877f5
9,How do I pass metadata in with @traceable?,To pass metadata using the @traceable function...,None,You can pass metadata with the @traceable deco...,1,1.454973,b256575c-e03c-44a4-bff6-64d4cbbeb65d,58e7d41a-6978-4557-a766-f086ce946faf


In [14]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o-mini"
)

View the evaluation results for experiment: 'gpt-4o-mini-3c8d3d53' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=b1a7d5ff-70c1-44c6-9833-c3508161ec5a




10it [01:11,  7.17s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.015870,16aa87a2-57b8-433e-b963-e2542c3642af,7135b69a-b08e-4847-81f0-4af7b5449c51
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.524291,1e9b198a-757d-4825-971d-37936c5bba30,9da3ada9-a251-429f-b542-1f6b6347d9d0
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.516354,341606a3-751a-41d4-8d13-2ae8cc502806,527fd3b5-3797-48d3-9070-5e6d3fabbf68
3,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,2.432133,59544459-9d57-431e-b2ee-e09f37ee1312,8e8f6644-c190-4d06-bcda-4da493e05263
4,What is LangSmith used for in three sentences?,LangSmith is used for managing and running exp...,None,LangSmith is a platform designed for the devel...,1,2.294114,5e199ead-d9d8-4f7a-a3bc-8365798aedf7,08ecf93d-abd9-4a15-ac25-e5e81e540724
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,45.963060,5efce925-42f9-45a4-82f7-71b43f10611f,2c8a3cda-5c57-4d6a-b520-29e6c321dfda
6,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,5.167479,61779256-72cc-45ad-a9fc-3471826ab829,bd1b8c4b-5e89-437f-b3e6-2a619c294ef0
7,Does LangSmith support offline evaluation?,The context provided does not mention offline ...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.644592,6d392d5b-1b24-4332-87a1-4a696cd36c07,47630c85-a45c-4080-bca9-6631138a66d3
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily a platform for observab...,None,"Yes, LangSmith can be used for fine-tuning and...",1,3.859143,90675349-ce91-4d9c-aef9-b186912f90ba,af9ae3a1-bb85-4330-95a1-b02d4b1f0099
9,How do I pass metadata in with @traceable?,You can pass metadata in with `@traceable` by ...,None,You can pass metadata with the @traceable deco...,1,2.773510,b256575c-e03c-44a4-bff6-64d4cbbeb65d,1169ce10-1517-4862-9745-c13352814959


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [50]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="Initial"),
    evaluators=[is_concise_enough],
    experiment_prefix="Initial dataset"
)

View the evaluation results for experiment: 'Initial dataset-7e9909d2' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=90253dc7-b51d-4038-a007-9025bfa247fc




10it [01:21,  8.10s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.865411,16aa87a2-57b8-433e-b963-e2542c3642af,44545260-fc1b-4059-b155-6f5fdfc74e85
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.612303,1e9b198a-757d-4825-971d-37936c5bba30,d388ff3b-5197-4571-9e98-a7b0682e3093
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,65.177962,341606a3-751a-41d4-8d13-2ae8cc502806,41ed1e62-ac5c-41c9-a337-3b07912b8f3b
3,What testing capabilities does LangSmith have?,LangSmith in LangChain provides testing capabi...,None,LangSmith offers capabilities for creating dat...,1,1.832240,59544459-9d57-431e-b2ee-e09f37ee1312,236d6bcf-7581-4610-b708-8a35bb65cb3c
4,What is LangSmith used for in three sentences?,LangSmith is used for running experiments on a...,None,LangSmith is a platform designed for the devel...,1,1.664292,5e199ead-d9d8-4f7a-a3bc-8365798aedf7,d6162b66-f307-4d01-85ef-9cd03f98cfe7
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,2.119718,5efce925-42f9-45a4-82f7-71b43f10611f,892034ca-f2d9-4b58-9e0d-fe8a9ed5002a
6,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.746087,61779256-72cc-45ad-a9fc-3471826ab829,8dd5c0de-1db3-416e-a957-f4189d2bd3d9
7,Does LangSmith support offline evaluation?,LangSmith does not support offline evaluation ...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.406432,6d392d5b-1b24-4332-87a1-4a696cd36c07,0d68ebd6-138c-4f51-9523-5473191e1c71
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.417749,90675349-ce91-4d9c-aef9-b186912f90ba,0a674a45-c75f-461d-97b5-f293dfed9bd7
9,How do I pass metadata in with @traceable?,You can pass metadata with @traceable by addin...,None,You can pass metadata with the @traceable deco...,1,1.649886,b256575c-e03c-44a4-bff6-64d4cbbeb65d,241a2996-a3a0-411b-ba0d-4127c579710f


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [34]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Exam"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial"
)


View the evaluation results for experiment: 'Crucial-86e0eea5' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=cae8d84d-4a6c-45bf-b927-af052968095a




5it [00:44,  8.91s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,LangSmith supports online evaluation through i...,None,"Yes, LangSmith supports online evaluation as a...",0,1.888512,16aa87a2-57b8-433e-b963-e2542c3642af,e4cd8698-5cf5-423c-91f1-681687d34606
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.411790,341606a3-751a-41d4-8d13-2ae8cc502806,f325c214-91f7-4781-9daa-75ecd61f5f2c
2,What is LangSmith used for in three sentences?,LangSmith is used for running multiple experim...,None,LangSmith is a platform designed for the devel...,1,2.210627,5e199ead-d9d8-4f7a-a3bc-8365798aedf7,5629345f-8ce6-49cc-bfb0-e2cb46186c67
3,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ca...",None,To trace with the @traceable decorator in Pyth...,1,35.037085,61779256-72cc-45ad-a9fc-3471826ab829,3df3f4fa-a681-4a58-aa23-7458c41af39d
4,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,2.456308,b256575c-e03c-44a4-bff6-64d4cbbeb65d,6700a3ce-5da5-493e-9453-472839599030


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [51]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[
            "16aa87a2-57b8-433e-b963-e2542c3642af",
            "6d392d5b-1b24-4332-87a1-4a696cd36c07"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-6ebe17a1' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=d9356bea-ddb5-46bf-8970-9cdc09a703af




2it [00:03,  1.96s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.970172,16aa87a2-57b8-433e-b963-e2542c3642af,a916a33a-c314-4420-ad2c-5d8a06570873
1,Does LangSmith support offline evaluation?,LangSmith currently supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.426506,6d392d5b-1b24-4332-87a1-4a696cd36c07,62371a88-c08a-43a2-8790-eff3a6d5c06c


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [52]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-1ff81b79' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=360aec70-c4c2-49fa-bb05-ed1e9ec4c10b




20it [02:59,  9.00s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,LangSmith supports online evaluation by provid...,None,"Yes, LangSmith supports online evaluation as a...",0,1.749005,16aa87a2-57b8-433e-b963-e2542c3642af,ca588b98-cb30-49ee-95f3-d6151f0f14e6
1,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.429866,341606a3-751a-41d4-8d13-2ae8cc502806,4000c527-7363-4814-813c-b6c4515532b2
2,What is LangSmith used for in three sentences?,LangSmith is used for running multiple experim...,None,LangSmith is a platform designed for the devel...,1,1.412095,5e199ead-d9d8-4f7a-a3bc-8365798aedf7,391506b4-91a6-4473-927d-c7a66206fe07
3,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.457255,61779256-72cc-45ad-a9fc-3471826ab829,b62f6461-4e01-4cb5-a2f8-6cdc6b491df9
4,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can a...",None,You can pass metadata with the @traceable deco...,1,1.623539,b256575c-e03c-44a4-bff6-64d4cbbeb65d,b790bb44-44ed-4349-bedb-0beb8e6fac51
5,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,66.671255,1e9b198a-757d-4825-971d-37936c5bba30,b23c52f6-928d-4f2f-8ad3-9f1b3b07f66a
6,What testing capabilities does LangSmith have?,LangSmith in LangChain offers the capability t...,None,LangSmith offers capabilities for creating dat...,1,1.420845,59544459-9d57-431e-b2ee-e09f37ee1312,a5844e2b-cfcf-4e72-856e-b4f97518352d
7,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,3.199282,5efce925-42f9-45a4-82f7-71b43f10611f,53e43646-53f3-4fb1-bee2-43007bfd6f22
8,Does LangSmith support offline evaluation?,LangSmith supports online evaluations for prov...,None,"Yes, LangSmith supports offline evaluation thr...",1,22.175472,6d392d5b-1b24-4332-87a1-4a696cd36c07,e8051d43-7bfd-4a21-adc3-08b3df153306
9,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for building p...,None,"Yes, LangSmith can be used for fine-tuning and...",0,1.850581,90675349-ce91-4d9c-aef9-b186912f90ba,f7da73bc-56dd-441f-a1e5-5ba721cad2f5


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [53]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-1efbc173' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=7a96eb5d-3368-4267-ab46-acd690843dc9




10it [00:40,  4.10s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. The...",None,"Yes, LangSmith supports online evaluation as a...",0,1.498346,16aa87a2-57b8-433e-b963-e2542c3642af,effedf6b-517c-4848-a0f4-439642b5fa23
1,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.735689,341606a3-751a-41d4-8d13-2ae8cc502806,0526f3bc-4efc-4773-b423-0c7db47d5bd4
2,What is LangSmith used for in three sentences?,LangSmith is used as an application for runnin...,None,LangSmith is a platform designed for the devel...,1,1.851598,5e199ead-d9d8-4f7a-a3bc-8365798aedf7,f3f0f454-7a10-4dd8-8e07-a8ecb2f18790
3,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.426124,61779256-72cc-45ad-a9fc-3471826ab829,ac76e9c9-a619-4e1a-bf61-c77d53f604a1
4,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.501598,1e9b198a-757d-4825-971d-37936c5bba30,073a8fab-f784-401a-871e-038c638a4d75
5,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.531524,59544459-9d57-431e-b2ee-e09f37ee1312,0acbe333-eeda-45d3-ac32-29b092974bf8
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.297037,5efce925-42f9-45a4-82f7-71b43f10611f,b834e2b3-b145-4e2e-aadb-5c19014d1c7e
7,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.374224,6d392d5b-1b24-4332-87a1-4a696cd36c07,5f5600b4-8fd3-48f9-a94b-155114456483
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.601826,90675349-ce91-4d9c-aef9-b186912f90ba,576ae207-9f18-4538-96b6-cb7c78629fab
9,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can i...",None,You can pass metadata with the @traceable deco...,1,38.768455,b256575c-e03c-44a4-bff6-64d4cbbeb65d,514def39-5d68-4558-8491-f66f47dd9e70


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [54]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-34616d40' at:
https://smith.langchain.com/o/bf0fc575-5ea9-4d25-adae-f03518e40c65/datasets/54b90069-d3a1-4023-ba54-2719faf57087/compare?selectedSessions=3d0f9174-898a-4152-83f4-021a247cdd6f




10it [00:22,  2.27s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.460443,16aa87a2-57b8-433e-b963-e2542c3642af,7789f0d3-a02b-4f0c-86e0-abca7be5c820
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.934577,341606a3-751a-41d4-8d13-2ae8cc502806,20e1b2db-4c50-40b6-adbd-cef0eefe2591
2,What is LangSmith used for in three sentences?,LangSmith is used for running multiple experim...,None,LangSmith is a platform designed for the devel...,1,2.062944,5e199ead-d9d8-4f7a-a3bc-8365798aedf7,d2c62cc6-1f36-4834-be3c-6c75a0c2ce44
3,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Lang...,None,To trace with the @traceable decorator in Pyth...,1,6.179693,61779256-72cc-45ad-a9fc-3471826ab829,720d5d9b-0436-46bf-8d90-fb455a73925f
4,How do I pass metadata in with @traceable?,"To pass metadata along with `@traceable`, you ...",None,You can pass metadata with the @traceable deco...,1,2.003038,b256575c-e03c-44a4-bff6-64d4cbbeb65d,8b6d3256-fef7-494a-8b6e-d9d8ad875a68
5,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.416288,1e9b198a-757d-4825-971d-37936c5bba30,c9a9c62f-214d-4b91-9155-52c2f423877f
6,What testing capabilities does LangSmith have?,LangSmith in LangChain provides testing capabi...,None,LangSmith offers capabilities for creating dat...,1,1.810401,59544459-9d57-431e-b2ee-e09f37ee1312,ff2b8260-9ef6-4346-96ca-de87599f8789
7,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.011747,5efce925-42f9-45a4-82f7-71b43f10611f,dba8802d-5d3c-45c1-9096-7b953c4c95a3
8,Does LangSmith support offline evaluation?,LangSmith supports online evaluations for real...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.558041,6d392d5b-1b24-4332-87a1-4a696cd36c07,53171787-0b70-4a5f-85b8-7b9625c6051d
9,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for observabil...,None,"Yes, LangSmith can be used for fine-tuning and...",0,1.698345,90675349-ce91-4d9c-aef9-b186912f90ba,a142e253-2fc3-4d51-8a7e-59d0bc6597a8
